In [1]:
import pandas as pd

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [22]:
ds = pd.read_csv('../data/r_1_SPB_clear_w_time_to_m.csv',index_col=0)

In [23]:
ds_reg = ds.drop(['a_lat','a_long','m_lat','m_long','adress','Стиральная машина','district','id','underground'],axis = 1)
ds_reg = ds_reg.drop(ds_reg[ds_reg['Балкон/лоджия'].isin(['2 лоджии','2 балкона'])].index)
ds_reg = ds_reg.drop(ds_reg[ds_reg['time_to_rent'] == 0].index) #удалим те, что ушли за 0 дней, так как большинство из них - то, что спрасили сегодня
ds_reg.reset_index(inplace=True, drop=True)

In [24]:
ds_reg['time_to_metro'] = ds_reg['time_to_metro'].apply(lambda x: round(x))
ds_reg['first_floor'] = ds_reg['floor'].apply(lambda x: 1 if x == 1 else 0)

In [25]:
ds_shuffled = ds_reg.sample(len(ds_reg),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds)*0.85)]
ds_test = ds_shuffled[int(len(ds)*0.85):]

In [26]:
Y_train = ds_train['time_to_rent']
X_train = ds_train.drop('time_to_rent',axis = 1)

In [27]:
Y_test = ds_test['time_to_rent']
X_test = ds_test.drop('time_to_rent',axis = 1)

In [28]:
bin_cols = list(X_train.nunique()[X_train.nunique() == 2].index)
bin_cols

['Холодильник',
 'Телевизор',
 'Посудомоечная машина',
 'Кондиционер',
 'Интернет',
 'Санузел',
 'first_floor']

In [29]:
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
num_cols

['floor',
 'floors_count',
 'total_meters',
 'price_per_month',
 'Площадь кухни',
 'Высота потолков',
 'Год постройки',
 'time_to_metro']

In [30]:
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)
cat_cols

['Балкон/лоджия', 'Вид из окон', 'Ремонт', 'Тип дома', 'Парковка']

In [31]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [32]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((692, 34), (41, 34))

In [33]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])

In [35]:
X_train = pd.DataFrame(X_train_norm,columns=col_names)
X_test = pd.DataFrame(X_test_norm,columns=col_names)

In [36]:
import statsmodels.api as sm

In [37]:
X_train = sm.add_constant(X_train,has_constant='add')
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [38]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 34, but rank is 30
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           time_to_rent   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     39.64
Date:                Mon, 04 Mar 2024   Prob (F-statistic):          9.22e-127
Time:                        03:05:52   Log-Likelihood:                -1877.9
No. Observations:                 692   AIC:                             3816.
Df Residuals:                     662   BIC:                             3952.
Df Model:                          29                                         
Covariance Type:                  HC3                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               19.2602     11.720      1.643      0.100      -3.711      42.231
Балкон/лоджия_1 балкон               4.9260      2.941      1.675      0.094      -0.838      10.690
Балкон/лоджия_1 лоджия               4.8156      2.961      1.626      0.104      -0.988      10.619
Балкон/лоджия_1 лоджия, 1 балкон     4.2709      3.140      1.360      0.174      -1.883      10.425
Балкон/лоджия_нет балкона            5.2477      2.868      1.830      0.067      -0.373      10.868
Вид из окон_Во двор                  6.5246      3.956      1.649      0.099      -1.228      14.277
Вид из окон_На улицу                 6.5924      3.907      1.688      0.092      -1.064      14.249
Вид из окон_На улицу и двор          6.1432      3.912      1.570      0.116      -1.524      13.810
Ремонт_Дизайнерский                  7.0062      3.908      1.793      0.073      -0.654      14.666
Ремонт_Евроремонт                    5.9972      3.957      1.516      0.130      -1.757      13.752
Ремонт_Косметический                 6.2568      3.889      1.609      0.108      -1.365      13.878
Тип дома_Блочный                     3.6075      2.388      1.511      0.131      -1.072       8.287
Тип дома_Кирпичный                   3.9202      2.298      1.706      0.088      -0.585       8.425
Тип дома_Монолитно-кирпичный         3.9786      2.402      1.656      0.098      -0.729       8.686
Тип дома_Монолитный                  4.1293      2.414      1.711      0.087      -0.602       8.861
Тип дома_Панельный                   3.6246      2.374      1.527      0.127      -1.028       8.277
Парковка_Многоуровневая              4.4810      3.004      1.492      0.136      -1.406      10.368
Парковка_Наземная                    4.9380      2.874      1.718      0.086      -0.695      10.571
Парковка_Открытая                    5.9166      3.022      1.958      0.050      -0.006      11.839
Парковка_Подземная                   3.9246      2.948      1.331      0.183      -1.853       9.702
floor                                0.0050      0.034      0.145      0.885      -0.062       0.072
floors_count                         0.0697      0.038      1.815      0.070      -0.006       0.145
total_meters                         0.0194      0.036      0.544      0.586      -0.050       0.089
price_per_month                   4.815e-05   2.27e-05      2.120      0.034    3.63e-06    9.27e-05
Площадь кухни                       -0.0129      0.057     -0.228      0.820      -0.124       0.098
Высота потолков                      0.9959      1.606      0.620      0.535      -2.151       4.143
Год постройки                       -0.0238      0.014     -1.658      0.097      -0.052       0.004
time_to_metro                        0.0262      0.00

In [40]:
X_test = sm.add_constant(X_test,has_constant='add')
preds_ts = results.predict(X_test)

In [43]:
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(Y_test,preds_ts)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(Y_test,preds_ts)))
print('Тестовый r2: {}'.format(r2_score(Y_test,preds_ts)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(Y_test,preds_ts)/Y_test.mean())*100,2)))
print('Тестовое среднее время: {}'.format(Y_test.mean()))
print('Предсказанное среднее время: {}'.format(preds_ts.mean()))

Тестовая средння абсолютная ошибка: 3.2487277000891925
Тестовая среднеквадратичная ошибка: 16.102886550129654
Тестовый r2: -0.10260498129401041
Тестовое отношение mae к среднему: 64.04%
Тестовое среднее время: 5.073170731707317
Предсказанное среднее время: 4.93167760149512


---

### Результат ужасный, так как данные не отражают действительности, так как прошло слишком мало дней


я дальше трогать ничего не буду, так как пока что, стат значимость переменных ни о чем не говорит

In [21]:
ds = ds.drop(['Санузел'],axis = 1)

In [22]:
ds_shuffled = ds.sample(len(ds),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds)*0.85)]
ds_test = ds_shuffled[int(len(ds)*0.85):]

In [23]:
Y_train = ds_train['time_to_rent']
X_train = ds_train.drop('time_to_rent',axis = 1)
Y_test = ds_test['time_to_rent']
X_test = ds_test.drop('time_to_rent',axis = 1)

In [24]:
bin_cols = list(ds.nunique()[ds.nunique() == 2].index)
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [26]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((941, 84), (314, 84))

In [27]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])
X_train = pd.DataFrame(X_train_norm.toarray(),columns=col_names)
X_test = pd.DataFrame(X_test_norm.toarray(),columns=col_names)

In [28]:
X_train = sm.add_constant(X_train)
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [29]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 84, but rank is 79
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           time_to_rent   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     18.00
Date:                Mon, 26 Feb 2024   Prob (F-statistic):          2.21e-134
Time:                        06:53:15   Log-Likelihood:                -2081.9
No. Observations:                 941   AIC:                             4322.
Df Residuals:                     862   BIC:                             4705.
Df Model:                          78                                         
Covariance Type:                  HC3                                         
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                      -0.1376      4.965     -0.028      0.978      -9.869       9.593
underground_Автово                         -0.4313      0.642     -0.672      0.502      -1.690       0.827
underground_Академическая                  -0.0696      0.458     -0.152      0.879      -0.968       0.829
underground_Балтийская                      0.9132      1.245      0.733      0.463      -1.527       3.354
underground_Беговая                         0.3690      0.399      0.924      0.356      -0.414       1.152
underground_Василеостровская                0.6879      0.875      0.786      0.432      -1.028       2.403
underground_Выборгская                      1.3900      0.932      1.491      0.136      -0.438       3.218
underground_Гостиный двор                   1.4006      1.572      0.891      0.373      -1.680       4.481
underground_Гражданский проспект            0.1747      0.400      0.437      0.662      -0.609       0.958
underground_Девяткино                       0.3508      0.665      0.527      0.598      -0.954       1.655
underground_Дунайская                       0.9569      0.639      1.499      0.134      -0.295       2.209
underground_Елизаровская                   -0.3296      1.264     -0.261      0.794      -2.807       2.148
underground_Звездная                       -0.4121      0.437     -0.942      0.346      -1.269       0.445
underground_Комендантский проспект          0.4634      0.316      1.466      0.143      -0.156       1.083
underground_Купчино                         0.5680      0.583      0.974      0.330      -0.574       1.710
underground_Ладожская                      -1.0099      0.658     -1.535      0.125      -2.299       0.279
underground_Ленинский проспект              0.3030      0.343      0.882      0.378      -0.370       0.976
underground_Лесная                         -0.1451      0.489     -0.297      0.767      -1.104       0.814
underground_Лиговский проспект              0.5156      1.178      0.438      0.662      -1.793       2.824
underground_Ломоносовская                   0.4140      0.664      0.623      0.533      -0.888       1.716
underground_Международная                   0.5158      0.776      0.665      0.506      -1.005       2.037
underground_Московская                      0.1619      0.499      0.324      0.746      -0.816       1.140
underground_Московские ворота              -0.8907      1.256     -0.709      0.478      -3.353       1.572
underground_Нарвская                       -0.8617      0.535     -1.611      0.107      -1.910       0.187
underground_Новочеркасская                 -0.5681      0.448     -1.268      0.205      -1.446       0.310
underground_Обводный канал                  0.7596      0.6

In [ ]:
X_test = sm.add_constant(X_test)
preds_ts = results.predict(X_test)

In [30]:
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(Y_test,preds_ts)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(Y_test,preds_ts)))
print('Тестовый r2: {}'.format(r2_score(Y_test,preds_ts)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(Y_test,preds_ts)/Y_test.mean())*100,2)))
print('Тестовая средняя цена: {}'.format(Y_test.mean()))
print('Предсказанная средняя цена: {}'.format(preds_ts.mean()))

Тестовая средння абсолютная ошибка: 1.8905314334211432
Тестовая среднеквадратичная ошибка: 5.600309572209118
Тестовый r2: -0.08001432261770947
Тестовое отношение mae к среднему: 71.01%
Тестовая средняя цена: 2.662420382165605
Предсказанная средняя цена: 2.7012679575123983


#### Удаление района ситуацию не сильно изменило, хотя по идее это примерно тоже самое, что и метро